In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
import os


# Step 1: Prepare the knowledge base
loader = PyMuPDFLoader("data/RAMAYANA.pdf")
documents = loader.load()

# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

file = open('data/key.txt', 'r')
# Read the API key from the file
api_key = file.read().strip()
# Close the file
file.close()
# 🧠 Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = api_key



In [ ]:
# Embeddings + FAISS Vectorstore
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                        model_kwargs={"device": "cpu"})
vectorstore = FAISS.from_documents(texts, embedding)

# Step 2: Setup memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"  # Specify which output to store in memory
)


In [13]:
# Step 3: Initialize LLM and Retrieval Chain with memory
llm = ChatOpenAI(temperature=0, model="gpt-4")

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    return_source_documents=True
)



In [14]:
# Step 4: Run queries
query1 = "Who is Rama?"
result1 = qa_chain({"question": query1})
print("Response 1:", result1["answer"])

query2 = "Tell me about his father?"
result2 = qa_chain({"question": query2})
print("Response 2:", result2["answer"])

Response 1: Rama, also known as Ramachandra, is a major deity in Hinduism. He is the seventh avatar of the god Vishnu and is also considered to be a supreme being in his own right. Rama is the protagonist of the Hindu epic Ramayana, which narrates his idealistic virtues and his adventures. In the provided context, Rama is depicted as a prince who is committed to fulfilling his father's promises, even at the cost of his own comfort and happiness.
Response 2: The father of Rama, the major deity in Hinduism, is King Dasharatha.
